In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#! pip install git+https://github.com/beroguedou/SpecAugment.git
#!pip install torchaudio

In [3]:
import os
import csv
import time
import librosa

import warnings
import pandas as pd
from models import *
from utils import *
from decode import *
import torch
import torchaudio
import numpy as np
import torch.optim as optim
from transformers import BertTokenizer

warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
%%time 

limit = 80
params = {'batch_size': 20,
          'shuffle': True,
          'num_workers': 10,
          'drop_last': True}


#tokenizer = tokenizer_librispeech(limit=800)
training_set = LibriSpeechDataset(limit=limit, n_frames=600, version='train-clean-360')
dev_set = LibriSpeechDataset(limit=limit, n_frames=600, version='dev-clean')

train_dataloader = data.DataLoader(training_set, **params)
dev_dataloader = data.DataLoader(dev_set, **params)

steps_per_epoch = len(training_set) // params['batch_size']

#vocab_tar_size = len(tokenizer.word_index)
embedding_dim_target = 100

CPU times: user 353 ms, sys: 45.4 ms, total: 399 ms
Wall time: 1.14 s


In [5]:
# Let's see an utterance

LibriSpeechDataset(limit=limit, n_frames=600, version='train-clean-360')[0]

(tensor([[[2.9673e-05, 3.7982e-05, 5.5997e-05,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [3.7324e-05, 9.5523e-01, 4.6501e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [3.3766e+00, 2.9255e+00, 1.0153e+01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]]]),
 tensor([  101,   101,  2045,  2001,  2625,  1997,  1037,  4306,  2055,  2085,
          1998, 14163,  6894,  5289,  2106,  2025,  2031,  2000,  2562,  2067,
          1037,  5481,  2004,  2002,  2441,  1996,  9445,  3460,  6031,  2988,
          1996, 14460,  4214,  8721,  2000, 10767,  1998, 27468,  1996,  2221]))

In [6]:
encoder = EncoderCONV2DRNN(batch_size=params['batch_size'], device=device, hidden_size=64).to(device)

decoder = DecoderATTRNN(30000, dec_units=64, batch_sz=params['batch_size'], 
                         hidden_size=64).to(device)


encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [8]:
# Train the model
global_trainer(1, train_dataloader, dev_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer,
                                    criterion, device, params['batch_size'],  bad_audio_mode=True)

 ====== ====== ====== ====== ====== ======
      The model has 4954139 parameters
 ====== ====== ====== ====== ====== ======


Epoch        1: 100%|████████████████████| 4/4 [01:07<00:00, 16.89s/it, Train loss 9.4923 Eval loss 8.8696]

Time taken for the training 0.018938 hours



In [11]:
mfccs, references = training_set[1]
tokenizer =  BertTokenizer.from_pretrained('bert-base-uncased')
references = [tokenizer.convert_ids_to_tokens(ind) for ind in references.numpy().tolist()]
evaluate(mfccs.unsqueeze(0), references, 40, encoder, decoder, targ_lang_tokenizer=tokenizer, 
          device=device, beam_search=True)

Input: ['[CLS]', '[CLS]', 'what', 'did', 'you', 'do', 'next', 'i', 'awakened', 'with', 'a', 'sudden', 'start', 'just', 'before', 'six', 'o', "'", 'clock', 'i', 'had', 'not', 'set', 'an', 'alarm', 'though', 'i', 'wanted', 'to', 'get', 'up', 'early', 'to', 'do', 'a', 'little', 'repair', 'job', 'i', 'had']


Predicted translation: ['[CLS]']


Bleu score: 1.722611814452713e-171


In [ ]:
# Vérifications

# 1 - Data Augmentation
# 2 - Encoder
# 3 - Attention Mechanism Bahdanau Audio
# 4 - Smoothing and Topk to the attention
# 5 - Decoder 
# 6 - Métrique BLEU

In [ ]:
# Faire un échéancier conforme à un des papiers
